<div align=center>
		
<p></p>
<p></p>
<font size=5>
In the Name of God
<font/>
 <p></p>
 <br/>
		<div align=center>
		    <size=3.5>
			    <br />
Practical Assignment 4- part 2
              <font  size=4>
            	<br/>
              N-grams
			</font>
    <br/>
    <br/>
 </div>
<hr/>
Artifical Intelligence - Dr. GholamReza GhassemSani
</font>
  <p></p>
 <br/>
Sharif University of Technology
<p></p>
Spring 2022


<br/>
<br/>
<font size=4 color=red>
 </font>
                <br/>
    </div>

##  Assignment Details
**Due date: 21/4/1401 (at 11:59pm)**<br>
You are free to collaborate but solutions must be written up individually.
Collaborators **must** be acknowledged.<br>
Submissions with more than 100 hours delay will not be graded.<br>Submissions with less than
50 hours delay will be penalized by the following rule:<br>
**Penalized mark = M * (100 – D) / 100** <br>
Where M = the mark achieved from your solution and D is number of hours passed the
deadline. Submissions within 50 < X ≤ 100 hours delay will be penalized by P.M. = M * 0.5.<br>
Submit your answers on quera.

student_number = 98110073<br>
Name = Farid<br>
Last_Name = Fotoohi

# Overview
In this assignment you will be implementing N-gram language models. N-gram language models assume that each word $wi$
depends only on the N−1 preceding words. you will be using these language models for:
*   calculating the probability of sentences
*   predicting next word of a query

###  Dataset
Download dataset from [here](https://drive.google.com/drive/folders/1OOjVMG61mrBmGDh2KgPrO0HqrMPj-M0j?usp=sharing).  
The dataset you will be working with for this assignment are available as `train.pickle` and `test.pickle` files. Each file includes a list of lists and each list contains tokens of a sentences/sentences.  
 Run the codes below to know more about the data 


In [1]:
import pickle

with open("data/train.pickle", 'rb') as handle:
    train_data = pickle.load(handle)

with open("data/test.pickle", 'rb') as handle:
    test_data = pickle.load(handle)

In the cell below we have a sample tokenized sentence

In [3]:
train_data[1]

['chillin',
 'on',
 'it',
 'waiting',
 'on',
 'the',
 'fam',
 'to',
 'get',
 'in',
 'wats',
 'good',
 'wit',
 'u']

# task 1 (Preprocessing)
At first explain what other preprocessing does the data need and and why then implement your suggested preprocessing.

*Explain your answers here*

In [4]:
import string
string.punctuation

def concat_texts(text):
    st = ""
    for i in text:
        if i != ' ':
            st += i +" "
    return st
    
def remove_punctuation(text):
    if(type(text)==float):
        return text
    ans=""  
    for i in text:     
        if i not in string.punctuation:
            ans+= i  
    return ans.strip()

In [5]:
train_data = [concat_texts(item) for item in train_data]
train_data = [remove_punctuation(item) for item in train_data]

In [6]:
train_data[1]

'chillin on it waiting on the fam to get in wats good wit u'

# Task 2 (Counting N-grams)
Fill out the following code block to count the unigrams, bigrams and trigrams in the data.


In [7]:
def generate_N_grams(text, ngram):
    words=[word for word in text.split(" ")]
    temp=zip(*[words[i:] for i in range(0,ngram)])
    ans=[' '.join(ngram) for ngram in temp]
    return ans

def LanguageModel(data):
    ### you need to use your preprocessed data and count the unigram,bigram and trigrams
    ### Begin your code
    unigram_counts = sum([len(generate_N_grams(item, 1)) for item in train_data])
    bigram_counts = sum([len(generate_N_grams(item, 2)) for item in train_data])
    trigram_counts = sum([len(generate_N_grams(item, 3)) for item in train_data])
    ### End your code
    ### return dictinaries with n-grams as keys and counts of n-grams as values
    return unigram_counts, bigram_counts, trigram_counts

In [8]:
unigram_counts, bigram_counts, trigram_counts = LanguageModel(train_data)

In [9]:
print(unigram_counts, bigram_counts, trigram_counts)

28839 26839 24839


# Task 3 (Laplace Smoothing)
In natural language:
*   A small number of events (e.g. words) occur with high frequency
*   large number of events occur with very low frequency\
So we can’t actually evaluate our MLE models on
unseen test data because both are likely to contain words/n-grams that these models assign zero probability to. In order to handle this problem we can use **1(laplace)smoothing** .  
Fill out the following code cell to calculate the probability of unigram, bigram and trigram.
  

In [8]:
from nltk import word_tokenize

In [9]:
def get_tokens(d):
    tokens = word_tokenize(d)
    return tokens

def get_tokens_freq(tkens):
    dct = dict()
    for item in tkens:
        dct.update({item:tkens.count(item)})
    return dct

tokens = list()
for i in train_data:
    tokens.extend(get_tokens(i))

for i in test_data:
    tokens.extend(i)

distinct_tokens = list(set(tokens))
token_freq_dct = get_tokens_freq(tokens)

In [10]:
def generate_ngram_freq(n_gram):
    dct = dict()
    distinct_ngrams = list(set(n_gram))
    for item in distinct_ngrams:
        dct.update({item:n_gram.count(item)})
    return dct

In [11]:
bigrams = list()
for i in train_data:
    bigrams.extend(generate_N_grams(i, 2))

bigram_freq_dct = generate_ngram_freq(bigrams)

In [12]:
trigrams = list()
for i in train_data:
    trigrams.extend(generate_N_grams(i, 3))

trigram_freq_dct = generate_ngram_freq(trigrams)

In [23]:
def bigram_probability(bigram_freq, token_freq):
    
    probability_dct = dict()
    
    for bigram in list(bigram_freq.keys()):
        items = bigram.split()
        if len(items)>1:
            probability_dct.update({bigram:bigram_freq[bigram]/token_freq[items[0]]})
    return probability_dct

def trigram_probability(trigram_freq, bigram_freq):
    
    probability_dct = dict()
    
    for trigram in list(trigram_freq.keys()):
        items = trigram.split()
        if len(items)>2:
            bigram = items[0] + " " + items[1]
            probability_dct.update({trigram:trigram_freq[trigram]/bigram_freq[bigram]})
    return probability_dct

In [19]:
bigram_probabilities = bigram_probability(bigram_freq_dct, token_freq_dct)

In [24]:
trigram_probabilities = trigram_probability(trigram_freq_dct, bigram_freq_dct)

In [26]:
# do not forget to add 1(laplace)smoothing or othe methods of smoothing(which you need to explain in another markdown cell)
def calc_unigram_p(w_1): # p(w_1)
    ### Begin your code
    p = token_freq_dct[w_1]/sum(list(token_freq_dct.values()))
    ### End your code
    return p
    
def calc_bigram_p(w_1,w_2): # p(w_2|w_1)
    ### Begin your code
    bigram = w_1 +  " " + w_2
    try:
        p =  bigram_probabilities[bigram]
    except:
        p = 1e-3
    ### End your code
    return p

def calc_trigram_p(w_1,w_2,w_3): # p(w_3|w_2,w_1)
    ### Begin your code
    trigram = w_1 + " " + w_2 +" "+ w_3
    try:
        p =  trigram_probabilities[trigram]
    except:
        p = 1e-3
    ### End your code
    return p

# Task 4 (Probability of sentences)
At this part first you need to choose 2 sentence(with more than 4 words) from the test data and  calculate the probability of the chosen sentences under each language model(unigram,bigram and trigram) as expplained below:
  
We will treat each sentence as a sequence of terms $(w_1, \ldots, w_n)$ whose probability under unigram model is computed as :
$$
P(w_1, \ldots, w_n) = P(w_1)P(w_2)\cdots P(w_n),
$$
under bigram model is computed as:
$$
P(w_1, \ldots, w_n) = P(w_1)P(w_2\mid w_1)\cdots P(w_n\mid w_{n-1}),
$$
and under trigram model is computed as:
$$
P(w_1, \ldots, w_n) = P(w_1)P(w_2\mid w_1)P(w_3\mid w_1,w_2)\cdots P(w_n\mid w_{n-2},w_{n-1}),
$$



In [27]:
    ### Begin your code
    
    def sent_prob(text):
        p = concat_texts(text)
        p = remove_punctuation(p)
        print('Sentence:',p, '\n')
        
        un = [calc_unigram_p(i) for i in p.split()]
        pun = 1
        for i in un:
            pun *= i
            
        bi = generate_N_grams(p, 2)
        pbi = calc_unigram_p(p.split()[0])
        for i in bi:
            j = i.split()
            if len(j)>1:
                pbi *= calc_bigram_p(j[0], j[1])
        
        tr = generate_N_grams(p, 3)
        ptr = calc_unigram_p(p.split()[0])
        bi = generate_N_grams(p, 2)
        ptr += calc_bigram_p(bi[0].split()[0], bi[0].split()[1])
        for i in tr:
            j = i.split()
            if len(j)>2:
                ptr *= calc_trigram_p(j[0], j[1], j[2])
        
        print(f"prob in: \n\tuni-gram:{pun}\n\tbi-gram:{pbi}\n\ttri-gram:{ptr}")
    
    ### End your code

In [28]:
sent_prob(test_data[0])

Sentence: seems difficult to find people on twitter  found nytimes  bbcscitech  but mainly hit and miss  any suggestions for finding content sources 

prob in: 
	uni-gram:3.70763003254425e-71
	bi-gram:1.3908131701819502e-50
	tri-gram:1.3944084222268704e-40


In [29]:
sent_prob(test_data[10])

Sentence: get your brunch on before you head out to summerfest   free shuttle starting at 4pm 

prob in: 
	uni-gram:4.251747805419834e-46
	bi-gram:1.184512760981032e-35
	tri-gram:4.0786446350135966e-36


# Task 5 (Finding The 3 Most Probable Next Words)
In this part you are given a query including 2 words.  
Query = i am  
You need to find the 2 words that are more likely to be the Word that follows `am` under each language model.

In [30]:
def get_similar_tokens(token, ngram_keys):
    temp = list()
    for item in ngram_keys:
        if item.startswith(token):
            temp.append(item)
    return temp

In [42]:
    ### Begin your code
    import copy
    import numpy as np
    def most_prob(sent):
        q = sent.split()
        
        # unigram model
        dct = {}
        for i in distinct_tokens:
            dct.update({i:calc_unigram_p(i)})
        key_list = list(dct.keys())
        val_list = list(dct.values())
        sort_list = copy.deepcopy(val_list)
        sort_list.sort(reverse=True)
        pos1 = val_list.index(sort_list[0])
        pos2 = val_list.index(sort_list[1])
        pos3 = val_list.index(sort_list[2])
        uni = [key_list[pos1], key_list[pos2], key_list[pos3]]
        
        
        # bigram model
        bigrams = list(bigram_probabilities.keys())
        temp = q[1]
        tokens = list()
        for i in range(3):
            t = get_similar_tokens(temp, bigrams)
            probs = [bigram_probabilities[i] for i in t]
            max_index = np.argmax(probs)
            temp = t[max_index].split()[-1]
            tokens.append(temp)
            
        # trigram model
        trigrams = list(trigram_probabilities.keys())
        temp = sent
        tr_tokens = list()
        for i in range(3):
            t = get_similar_tokens(temp, trigrams)
            probs = [trigram_probabilities[i] for i in t]
            max_index = np.argmax(probs)
            temp = t[max_index].split()[-1]
            tr_tokens.append(temp)
            temp = t[max_index].split()[1] + " " +t[max_index].split()[2]
        
        
        print(f"Words in:\n\tuni-gram Model--> {uni}\n \tBi-gram Model--> {tokens}\n\tTri-gram Model--> {tr_tokens}")
        
    ### End your code

In [43]:
most_prob('on the')

Words in:
	uni-gram Model--> ['sb', 'sb', 'sb']
 	Bi-gram Model--> ['immediately', 'despite', 'my']
	Bi-gram Model--> ['swag', 'then', 'their']


### Note that :
***Your answers will be evaluated based on correct implementation of the above functions.**  
***Each task has 20 points**
# The End